In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from data_processing import LoadData
from model import *
import syft as sy
import copy
hook = sy.TorchHook(torch)
import pandas as pd
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Yash\AppData\Roaming\Python\Python36\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:
def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    
    dataset = pd.read_csv('PeMS_04/PeMS04.csv')
    dataset_size = len(dataset)

    indices = list(range(dataset_size))
    np.random.shuffle(indices)

    split1 = int(np.floor(0.6 * dataset_size)) #60-40 Bob-Alice
    bobs_indices, alices_indices = indices[:split1], indices[split1:]

    bobs_sampler = SubsetRandomSampler(bobs_indices)
    alices_sampler = SubsetRandomSampler(alices_indices)

    train_data = LoadData(data_path=["PeMS_04/PeMS04.csv", "PeMS_04/PeMS04.npz"], num_nodes=307, divide_days=[45, 14],
                              time_interval=5, history_length=6,
                              train_mode="train")


    bobs_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=bobs_sampler)
    alices_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=alices_sampler)

#     # Loading Dataset
#     train_data = LoadData(data_path=["PeMS_04/PeMS04.csv", "PeMS_04/PeMS04.npz"], num_nodes=307, divide_days=[45, 14],
#                           time_interval=5, history_length=6,
#                           train_mode="train")

#     train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=32)
#....................................................
#     test_data = LoadData(data_path=["PeMS_04/PeMS04.csv", "PeMS_04/PeMS04.npz"], num_nodes=307, divide_days=[45, 14],
#                          time_interval=5, history_length=6,
#                          train_mode="test")

#     test_loader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=32)
    
#     for data in train_loader: 
#         print(data)

    #model = GCN(in_c=6 , hid_c=6 ,out_c=1)
    #ChebNet(in_c=6, hid_c=32, out_c=1, K=2)
    #GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bobs_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)     # 2阶切比雪夫模型    
    bobs_model = bobs_model.to(device)
    
    alices_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)      # 2阶切比雪夫模型
    alices_model = alices_model.to(device)

    #new
    bob = sy.VirtualWorker(hook, id="bob")
    alice = sy.VirtualWorker(hook, id="alice")
    secure_worker = sy.VirtualWorker(hook, id="secure_worker")

    #new
#     bobs_model = model.copy().send(bob)
#     alices_model = model.copy().send(alice)

    criterion = nn.MSELoss()
    
    #new
    bobs_opt = optim.Adam(params=bobs_model.parameters())
    alices_opt = optim.Adam(params=alices_model.parameters())

    # Train model
    Epoch = 8

    bobs_model.train()
    alices_model.train()
    for epoch in range(Epoch):
    	#new
        epoch_loss_bob = 0.0
        epoch_loss_alice = 0.0

        start_time = time.time()
        for data in bobs_loader:  # ["graph": [B, N, N] , "flow_x": [B, N, H, D], "flow_y": [B, N, 1, D]]
            bobs_model.zero_grad()
            predict_value = bobs_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_bob += loss.item()
            loss.backward()
            bobs_opt.step()
            
        for data in alices_loader:
            alices_model.zero_grad()
            predict_value = alices_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_alice += loss.item()
            loss.backward()
            alices_opt.step()
            
        end_time = time.time()

        print("Epoch: {:04d}, Loss Bob: {:02.4f}, Loss Alice: {:02.4f}, Time: {:02.2f} mins".format(epoch, 1000 * epoch_loss_bob / len(train_data), 1000 * epoch_loss_alice / len(train_data),
                                                                          (end_time-start_time)/60))

In [3]:
if __name__ == '__main__':
    main()

Epoch: 0000, Loss Bob: 0.0381, Loss Alice: 0.0241, Time: 3.43 mins
Epoch: 0001, Loss Bob: 0.0374, Loss Alice: 0.0272, Time: 2.66 mins
Epoch: 0002, Loss Bob: 0.0388, Loss Alice: 0.0276, Time: 2.69 mins
Epoch: 0003, Loss Bob: 0.0397, Loss Alice: 0.0296, Time: 3.03 mins
Epoch: 0004, Loss Bob: 0.0352, Loss Alice: 0.0227, Time: 2.72 mins
Epoch: 0005, Loss Bob: 0.0379, Loss Alice: 0.0283, Time: 2.57 mins
Epoch: 0006, Loss Bob: 0.0347, Loss Alice: 0.0234, Time: 3.24 mins
Epoch: 0007, Loss Bob: 0.0383, Loss Alice: 0.0275, Time: 2.87 mins
